# Predicción completa

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import joblib

from datetime import datetime


In [ ]:
# Definiciones globales y funciones de transformación

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py
%run ../Lib/fn_transformaciones.py


In [ ]:
# Cargar archivo de tramos

archivo_tramos = f'{DIRECTORIO_DATOS}/DATOSTRAMOS_2023-08-06.csv'
df_tramos = pd.read_csv(archivo_tramos, low_memory=False)

print(df_tramos.info())


In [ ]:
# Cargar modelo

archivo_modelo = f'{DIRECTORIO_MODELOS}/modelo.joblib'
modelo = joblib.load(archivo_modelo)
print(modelo)


In [ ]:
# Cargar columnas

columnas_modelo = []
archivo_columnas = f'{DIRECTORIO_MODELOS}/COLUMNAS.txt'

with open(archivo_columnas, "r") as archivo:
    for linea in archivo:
        cadena = linea.strip()
        columnas_modelo.append(cadena)

print(columnas_modelo)


In [ ]:
# Generar set de datos

df_predecir = df_tramos.copy()

# Agregar columnas dummy para codificacion
df_predecir['CALIFICACION'] = 1 # Irrelevante, no se usa pero es requerido para transformacion
df_predecir['FECHA_CALIFICACION'] = datetime.now()

X_predecir = tx_codificar(tx_aplicar_transformaciones(df_predecir))

# Agregar columnas faltantes
for col in columnas_modelo:
    if col not in X_predecir.columns:
        print(f'Agregando {col}')
        X_predecir[col] = False

# Reordenar columnas
X_predecir = X_predecir[columnas_modelo]

# Obtener tramos asociados
ipids = df_predecir.loc[df_predecir.index.isin(X_predecir.index)]['IPID']

print(X_predecir.info())
#print(df_ipids.info())

In [ ]:
# Calcular probabilidades

probabilidades = modelo.predict_proba(X_predecir)
probabilidades_deterioro = probabilidades[:, 1]

df_probabilidades = pd.DataFrame({'IPID': ipids, 'PROBABILIDAD_DETERIORO': probabilidades_deterioro}).sort_values(by='PROBABILIDAD_DETERIORO', ascending=False) 

print(df_probabilidades)


In [8]:
# Guardar reporte

archivo_predicciones_completo = f'{DIRECTORIO_RESULTADOS}/PREDICCIONES_COMPLETAS.csv'

df_probabilidades.to_csv(archivo_predicciones_completo, index=False)

In [ ]:
sns.kdeplot(data=df_probabilidades, x='PROBABILIDAD_DETERIORO', fill=True)
plt.xlabel('Probabilidad de Deterioro')
plt.ylabel('Densidad')
plt.title('Gráfico de Densidad de Probabilidad de Deterioro')
plt.show()


In [ ]:
sns.boxplot(data=df_probabilidades, x='PROBABILIDAD_DETERIORO')
plt.xlabel('Probabilidad de Deterioro')
plt.title('Box Plot de Probabilidad de Deterioro')
plt.show()
